# DNAmPhenoAge

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.DNAmPhenoAge)

class DNAmPhenoAge(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        return x

    def postprocess(self, x):
        return x



In [3]:
model = pya.models.DNAmPhenoAge()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'dnamphenoage'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2018
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = "Levine, Morgan E., et al. \"An epigenetic biomarker of aging for lifespan and healthspan.\" Aging (albany NY) 10.4 (2018): 573."
model.metadata["doi"] = 'https://doi.org/10.18632%2Faging.101414'
model.metadata["research_only"] = None
model.metadata["notes"] = None

## Download clock dependencies

#### Download directly with curl

In [5]:
supplementary_url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5940111/bin/aging-10-101414-s002.csv"
supplementary_file_name = "coefficients.csv"
os.system(f"curl -o {supplementary_file_name} {supplementary_url}")

0

## Load features

#### From CSV file

In [6]:
df = pd.read_csv('coefficients.csv')
model.features = df['CpG'][1:].tolist()

## Load weights into base model

In [7]:
weights = torch.tensor(df['Weight'][1:].tolist()).unsqueeze(0).float()
intercept = torch.tensor([df['Weight'][0]]).float()

#### Linear model

In [8]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [9]:
model.reference_values = None

## Load preprocess and postprocess objects

In [10]:
model.preprocess_name = None
model.preprocess_dependencies = None

In [11]:
model.postprocess_name = None
model.postprocess_dependencies = None

## Check all clock parameters

In [12]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '⌛',
 'citation': 'Levine, Morgan E., et al. "An epigenetic biomarker of aging for '
             'lifespan and healthspan." Aging (albany NY) 10.4 (2018): 573.',
 'clock_name': 'dnamphenoage',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.18632%2Faging.101414',
 'notes': None,
 'research_only': None,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2018}
reference_values: None
preprocess_name: None
preprocess_dependencies: None
postprocess_name: None
postprocess_dependencies: None
features: ['cg15611364', 'cg17605084', 'cg26382071', 'cg12743894', 'cg19287114', 'cg12985418', 'cg19398783', 'cg15963417', 'cg27187881', 'cg09892203', 'cg00943950', 'cg18996776', 'cg16340918', 'cg23832061', 'cg22736354', 'cg04084157', 'cg07265300', 'cg02503970', 'cg11426590', 'cg23710218', 'cg02802055', 'cg13631913', 'cg06493994', 'cg24

## Basic test

In [13]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[ 134.9219],
        [ 396.0094],
        [-309.7721],
        [  77.3113],
        [-209.0913],
        [-141.9328],
        [ 424.4550],
        [-125.9833],
        [-119.9547],
        [-250.8203]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

## Save torch model

In [14]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [15]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: coefficients.csv
